# Project 2: Password Strength Checker

# Problem Statement:
Weak passwords are a major reason for data breaches. This project addresses the critical need for users to understand and improve the strength of their passwords.

# Objective:
To develop a user-friendly Graphical User Interface (GUI) tool that effectively analyzes the strength of a given password in real-time and provides actionable feedback and suggestions for improvement.

# Technologies Utilized:
* **Python:** The core programming language for the application logic.
* **Tkinter:** Python's standard GUI toolkit, used for building the interactive desktop application.
* **Regex (re module):** Employed for robust pattern matching to identify character types, common sequences, and repeated characters within passwords.
* **NLTK (Natural Language Toolkit) - Optional:** Integrated for advanced dictionary word detection, significantly enhancing the strength analysis by identifying easily guessable common words or phrases.

# Expected Outcome:
An intuitive and interactive desktop application that empowers users to assess their password's robustness. The tool will clearly indicate whether a password is  "Weak," "Moderate," "Strong," and offer specific recommendations to enhance its security.

In [3]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to C:\Users\SIVA RANJINI
[nltk_data]     R\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# Environment Setup and Library Imports

This section handles the necessary library imports for the project. It includes `re` for regular expressions, `tkinter` for the GUI, and optionally `nltk` for dictionary word detection. A check is performed to ensure the `nltk` 'words' corpus is available.

In [6]:
import re
from tkinter import Tk, Label, Entry, Button, StringVar, Text, END, W, E
from tkinter import messagebox
# Optional: For dictionary word checking
try:
    import nltk
    from nltk.corpus import words
    nltk_words = set(words.words()) # Load words into a set for fast lookup
    NLTK_AVAILABLE = True
except ImportError:
    NLTK_AVAILABLE = False
    print("NLTK not found. Dictionary word detection will be skipped.")
except LookupError:
    NLTK_AVAILABLE = False
    print("NLTK 'words' corpus not found. Please run 'nltk.download(\"words\")'. Dictionary word detection will be skipped.")


def check_password_strength(password):
    """
    Analyzes password strength based on various criteria and provides feedback.
    Returns a tuple: (strength_level, feedback_messages)
    Strength levels: 0 (Very Weak), 1 (Weak), 2 (Moderate), 3 (Strong), 4 (Very Strong)
    """
    score = 0
    feedback = []

    # Criteria 1: Length
    if len(password) >= 12:
        score += 2
        feedback.append("Good: Password is at least 12 characters long.")
    elif len(password) >= 8:
        score += 1
        feedback.append("Moderate: Password is at least 8 characters long.")
    else:
        feedback.append("Improvement: Password is too short. Aim for at least 8, ideally 12+ characters.")

    # Criteria 2: Character Variety
    has_lowercase = re.search(r'[a-z]', password)
    has_uppercase = re.search(r'[A-Z]', password)
    has_digit = re.search(r'\d', password)
    has_special = re.search(r'[!@#$%^&*(),.?":{}|<>]', password)

    char_types = 0
    if has_lowercase: char_types += 1
    if has_uppercase: char_types += 1
    if has_digit: char_types += 1
    if has_special: char_types += 1

    if char_types == 4:
        score += 2
        feedback.append("Excellent: Contains lowercase, uppercase, digits, and special characters.")
    elif char_types == 3:
        score += 1
        feedback.append("Good: Uses 3 out of 4 character types.")
    else:
        feedback.append("Improvement: Mix uppercase, lowercase, numbers, and symbols.")

    # Criteria 3: Avoid common patterns (e.g., sequences, repeated characters)
    if re.search(r'(.)\1\1', password): # Detects 3 or more repeated characters (e.g., "aaa")
        score -= 1
        feedback.append("Warning: Avoid using sequences of repeated characters.")

    if re.search(r'abc|def|123|xyz', password, re.IGNORECASE): # Detects common sequences
        score -= 1
        feedback.append("Warning: Avoid common sequences like 'abc' or '123'.")

    # Optional Criteria 4: Dictionary word detection (if NLTK is available)
    if NLTK_AVAILABLE:
        is_dictionary_word = False
        # Check if the whole password is a dictionary word (case-insensitive)
        if password.lower() in nltk_words:
            is_dictionary_word = True
        # Also check for common words within the password (e.g., "password123" -> "password")
        else:
            # Simple heuristic: Split by non-alphanumeric and check parts
            words_in_pass = re.findall(r'[a-zA-Z]+', password)
            for word_part in words_in_pass:
                if len(word_part) >= 4 and word_part.lower() in nltk_words: # Only check words >= 4 chars
                    is_dictionary_word = True
                    break

        if is_dictionary_word:
            score -= 2
            feedback.append("Major Warning: Password is a common dictionary word or contains one. Avoid common words.")
        else:
            feedback.append("Good: Does not appear to be a common dictionary word.")

    # Map score to strength level
    if score >= 5:
        strength_level = "Very Strong"
    elif score >= 3:
        strength_level = "Strong"
    elif score >= 1:
        strength_level = "Moderate"
    elif score >= 0:
        strength_level = "Weak"
    else:
        strength_level = "Very Weak"

    return strength_level, feedback

# Building the Graphical User Interface (GUI)

This section defines the `PasswordCheckerApp` class, which constructs the visual elements of the application using Tkinter. It sets up:
* The main window with a title and fixed size.
* An input field for the password, which masks characters (`show="*" `).
* A "Show/Hide" button to toggle password visibility.
* A label to display the password strength, with dynamic color-coding.
* A text box to provide detailed feedback and suggestions.

The class also includes methods to update the feedback text, toggle password visibility, and trigger the strength check every time the user types (`check_on_type`).

In [26]:
class PasswordCheckerApp:
    def __init__(self, master):
        self.master = master
        master.title("Password Strength Checker")
        master.geometry("500x400") # Set initial window size
        master.resizable(False, False) # Prevent resizing

        # --- Widgets ---

        # Label for password input
        self.password_label = Label(master, text="Enter Password:")
        self.password_label.grid(row=0, column=0, sticky=W, padx=10, pady=10)

        # Entry widget for password input (show="*" for masking)
        self.password_entry_var = StringVar()
        self.password_entry = Entry(master, show="*", width=40, textvariable=self.password_entry_var)
        self.password_entry.grid(row=0, column=1, padx=10, pady=10, sticky=W+E)
        self.password_entry_var.trace_add("write", self.check_on_type) # Check as user types

        # Check Button to toggle password visibility
        self.show_password_var = StringVar()
        self.show_password_var.set("Show")
        self.show_password_button = Button(master, textvariable=self.show_password_var, command=self.toggle_password_visibility)
        self.show_password_button.grid(row=0, column=2, padx=5, pady=10)

        # Strength Label
        self.strength_label = Label(master, text="Strength: ", font=("Arial", 12, "bold"))
        self.strength_label.grid(row=1, column=0, columnspan=3, sticky=W, padx=10, pady=5)

        # Feedback Text Box
        self.feedback_label = Label(master, text="Feedback & Suggestions:", font=("Arial", 10, "bold"))
        self.feedback_label.grid(row=2, column=0, columnspan=3, sticky=W, padx=10, pady=5)

        self.feedback_text = Text(master, height=10, width=50, wrap='word', state='disabled')
        self.feedback_text.grid(row=3, column=0, columnspan=3, padx=10, pady=5, sticky="nsew")

        # Configure grid to expand with window (though resizing is off, good practice)
        master.grid_rowconfigure(3, weight=1)
        master.grid_columnconfigure(1, weight=1)

        # Initial check (e.g., if there's default text, though usually empty)
        self.check_password_strength_gui()

    def update_feedback_text(self, messages):
        self.feedback_text.config(state='normal') # Enable editing
        self.feedback_text.delete(1.0, END) # Clear existing text
        for msg in messages:
            self.feedback_text.insert(END, f"- {msg}\n")
        self.feedback_text.config(state='disabled') # Disable editing

    def toggle_password_visibility(self):
        if self.password_entry.cget("show") == "*":
            self.password_entry.config(show="")
            self.show_password_var.set("Hide")
        else:
            self.password_entry.config(show="*")
            self.show_password_var.set("Show")

    def check_on_type(self, *args):
        # This function is called every time the user types in the password entry
        self.check_password_strength_gui()

    def check_password_strength_gui(self):
        password = self.password_entry_var.get()
        if not password:
            self.strength_label.config(text="Strength: ")
            self.update_feedback_text(["Please enter a password to check its strength."])
            return

        strength_level, feedback_messages = check_password_strength(password) # Call the core logic function

        # Update strength label based on result
        self.strength_label.config(text=f"Strength: {strength_level}",
                                   fg=self.get_strength_color(strength_level)) # Set color based on strength

        # Update feedback text box
        self.update_feedback_text(feedback_messages)

    def get_strength_color(self, strength_level):
        """Helper to get a color based on strength level."""
        if strength_level == "Very Strong":
            return "green"
        elif strength_level == "Strong":
            return "darkgreen"
        elif strength_level == "Moderate":
            return "orange"
        elif strength_level == "Weak":
            return "red"
        else: # Very Weak
            return "darkred"

# Running the Password Strength Checker Application
The following code block initiates the Tkinter main loop, which starts the GUI application. When executed, a separate desktop window will appear. 
Users can then interact with the application by typing passwords and observing real-time strength feedback and suggestions.

In [29]:
# --- Main Application Loop ---
# This block initiates and runs the Tkinter application.

if __name__ == "__main__":
    root = Tk() # Create the main window object
    app = PasswordCheckerApp(root) # Create an instance of your app, passing the main window
    root.mainloop() # Start the Tkinter event loop (makes the window appear and respond)

# Conceptual Output / Application Screenshot:

# Weak Password Example:
![Weak Password Screenshot](password_checker_weakpassword_img.png)

# Moderate Password Example:
![Moderate Password Screenshot](password_checker_moderatepassword_img.png)

# Strong Password Example:
![Strong Password Screenshot](password_checker_strongpassword_img.png)


# Conclusion - Project 2: Password Strength Checker

This project successfully developed an interactive GUI-based password strength checker using Python and Tkinter. The application effectively analyzes passwords based on length, character complexity, common patterns, and optionally dictionary words, providing immediate visual and textual feedback. This tool can serve as a practical utility for users aiming to improve their online security habits.